In [52]:
# Load environment variables from .env file
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


# Create Graph with a state of all messages

In [53]:
from langgraph.graph import add_messages, StateGraph
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.checkpoint.sqlite import SqliteSaver

class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]
    event_type: str
    location: str
    labels: str
    
graph_builder = StateGraph(State)

# Connection to Couchbase's vector store

In [54]:
from langchain_openai import OpenAIEmbeddings
import os
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions
from couchbase.auth import PasswordAuthenticator
from langchain_couchbase import CouchbaseVectorStore

COUCHBASE_CONNECTION_STRING = os.environ["COUCHBASE_CONNECTION_STRING"]
COUCH_USER = os.environ["COUCHBASE_USER"]
COUCH_PASS = os.environ["COUCHBASE_PASS"]
BUCKET_NAME = os.environ["COUCHBASE_BUCKET"]
SCOPE_NAME = os.environ["COUCHBASE_SCOPE"]
COLLECTION_NAME = os.environ["COUCHBASE_COLLECTION"]
SEARCH_INDEX_NAME = os.environ["COUCHBASE_SEARCH_INDEX"]

auth = PasswordAuthenticator(COUCH_USER, COUCH_PASS)
options = ClusterOptions(auth)
cluster = Cluster(COUCHBASE_CONNECTION_STRING, options)
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = CouchbaseVectorStore(
    cluster=cluster,
    bucket_name=BUCKET_NAME,
    scope_name=SCOPE_NAME,
    collection_name=COLLECTION_NAME,
    embedding=embedding,
    index_name=SEARCH_INDEX_NAME,
    embedding_key='embeddings'
)


# Create Graph instance

In [55]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")
template = """You are a helpful bot that serves the purpose of finding events for artists looking for venues in the USA. If you cannot answer based on the context provided, respond with a generic
    answer. Answer the question as truthfully as possible using the context below: {context}
Please also format the result in Markdown format.
            Question: {question}"""

prompt = ChatPromptTemplate.from_template(template)

generation_chain = prompt | llm | StrOutputParser()


def chatbot(state: State):
    response = generation_chain.invoke({"context": state['messages'], "question": f"We are a {state['event_type']} amateur group looking for {state['labels']} festivals in {state['location']}, can you please recommend some for us to reach out to?"})
    state['messages'].append(response)
    return state

def search_couchbase(state: State):
    query = f"""
    Geo Info: {state['location']}
    """
    
    retriever = vector_store.as_retriever()
    results = retriever.invoke(query)
    for result in results:
        text = f"Title: {result.metadata['title']}/{result.metadata['alternate_titles_flat']} - {result.metadata['description']} from {result.metadata['start']} to {result.metadata['end']}, location {result.metadata['geo_info']}. Labels {result.metadata['labels']}, category {result.metadata['category']}"
        state['messages'].append(text)
    return state

graph_builder.add_node("vector_search", search_couchbase)
graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("vector_search")
graph_builder.add_edge("vector_search", "chatbot")

memory = SqliteSaver.from_conn_string(":memory:")
graph = graph_builder.compile(checkpointer=memory)


# Interaction

In [58]:
from random import randint
from IPython.core.display import Markdown

session_id = randint(1, 10000)
config = {"configurable": {"thread_id": session_id}}

input_location = "Florida, USA"
input_category = "Music"
input_labels = "music"
for event in graph.stream({"event_type": input_category, "location": input_location, "labels": input_labels}, config):
    for value in event.values():
        if len(value['messages']) > 0:
            display(Markdown(value['messages'][-1]))

Title: Ashley Gavin (Theater)/ -  from 2024-10-04T23:00:00Z to 2024-10-06T01:00:00Z, location Tampa, FL 33605, United States of America. Labels ['comedy', 'entertainment', 'performing-arts'], category performing-arts

### Recommended Music Festivals in Florida, USA:

1. **Beaches Oktoberfest/Oktoberfest Jacksonville**
   - **Date:** October 6th to October 8th, 2024
   - **Location:** Florida, US
   - **Description:** A two-day, family-friendly festival with a variety of German beer, national performing acts, food trucks, a ferris wheel, arts & crafts vendors, games, and more.
   - **Category:** Festivals
   - [Event Link](insert event link)

2. **Michael Jr.'s Funny How Marriage Works Tour @ Tampa, FL**
   - **Date:** October 5th to October 6th, 2024
   - **Location:** Bradenton, FL 34211, United States of America
   - **Description:** Comedy Date-Night Experience by Michael Jr. focusing on marriage and dating in a unique way.
   - **Category:** Performing Arts
   - [Event Link](insert event link)

Feel free to reach out to these events for potential performance opportunities!